In [ ]:
!pip install -q transformers==4.30.0 datasets scikit-learn pandas > /dev/null 2>&1

import os
import warnings
import logging
import pandas as pd
from datasets import load_dataset, disable_progress_bar
from transformers import pipeline, logging as hf_logging

os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
warnings.filterwarnings("ignore")
hf_logging.set_verbosity_error()
disable_progress_bar()

from sklearn.metrics import accuracy_score, f1_score

print(">>> Loading Balanced Dataset (50 Toxic / 50 Non-toxic)...")

dataset_stream = load_dataset("google/civil_comments", split="train", streaming=True)

toxic_ds = list(dataset_stream.filter(lambda x: x['toxicity'] >= 0.5).take(50))
nontoxic_ds = list(dataset_stream.filter(lambda x: x['toxicity'] < 0.5).take(50))

dataset = toxic_ds + nontoxic_ds
data = pd.DataFrame(dataset)

data = data.sample(frac=1, random_state=42).reset_index(drop=True)

data['label'] = data['toxicity'].apply(lambda x: 1 if x >= 0.5 else 0)

print(f"Data Loaded: {len(data)} examples (Balanced: {data['label'].sum()} Toxic)")

def naive_baseline(text):
    text = text.lower()
    bad_words = ["stupid", "idiot", "dumb", "shut up", "trash", "kill", "ugly", "fat", "hell", "crazy"]
    if any(w in text for w in bad_words):
        return 1
    else:
        return 0

data['baseline_pred'] = data['text'].apply(naive_baseline)

print(">>> Running AI Pipeline (approx. 1-2 min)...")

toxicity_model = pipeline("text-classification", model="s-nlp/roberta_toxicity_classifier", device=-1)

def run_ai(text):
    truncated_text = text[:512]
    result = toxicity_model(truncated_text)[0]

    if result['label'] == 'toxic':
        return 1
    elif result['label'] == 'neutral':
        return 0
    else:
        return 1 if result['score'] > 0.5 else 0

data['ai_pred'] = data['text'].apply(run_ai)

baseline_acc = accuracy_score(data['label'], data['baseline_pred'])
ai_acc = accuracy_score(data['label'], data['ai_pred'])

baseline_f1 = f1_score(data['label'], data['baseline_pred'])
ai_f1 = f1_score(data['label'], data['ai_pred'])

print("\n" + "="*40)
print(f" [Results] Baseline Accuracy:   {baseline_acc:.2f}")
print(f" [Results] AI Pipeline Accuracy: {ai_acc:.2f}")
print("-" * 40)
print(f" [Results] Baseline F1 Score:   {baseline_f1:.2f}")
print(f" [Results] AI Pipeline F1 Score: {ai_f1:.2f}")
print("="*40 + "\n")


print(">>> [Failure Cases: Baseline Failed vs. AI Succeeded]")
diff = data[(data['baseline_pred'] != data['label']) & (data['ai_pred'] == data['label'])]

for i, row in diff.head(3).iterrows():
    print(f"\n[Case Index {i}]")
    print(f"Comment: \"{row['text'][:200]}...\"")
    print(f" - True Label:      {'Toxic' if row['label']==1 else 'Non-toxic'}")
    print(f" - Baseline Pred:   {'Toxic' if row['baseline_pred']==1 else 'Non-toxic'} (Incorrect)")
    print(f" - AI Pipeline Pred:{'Toxic' if row['ai_pred']==1 else 'Non-toxic'} (Correct)")

>>> Loading Balanced Dataset (50 Toxic / 50 Non-toxic)...


README.md: 0.00B [00:00, ?B/s]

Data Loaded: 100 examples (Balanced: 50 Toxic)
>>> Running AI Pipeline (approx. 1-2 min)...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]


 [Results] Baseline Accuracy:   0.61
 [Results] AI Pipeline Accuracy: 0.86
----------------------------------------
 [Results] Baseline F1 Score:   0.40
 [Results] AI Pipeline F1 Score: 0.84

>>> [Failure Cases: Baseline Failed vs. AI Succeeded]

[Case Index 3]
Comment: "They are terrorists pure and simple...."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)

[Case Index 5]
Comment: "I honestly cannot decide if these guys are complete morons or the most patriotic hero's this country has seen in a long time...."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)

[Case Index 8]
Comment: "Their ridiculous band photo has me wanting to drink white wine at 3 in the afternoon.
It's almost 5, right?..."
 - True Label:      Toxic
 - Baseline Pred:   Non-toxic (Incorrect)
 - AI Pipeline Pred:Toxic (Correct)
